In [13]:
import numpy as np
import bitarray

In [14]:
fileObject = open("input.txt", "r")
data = fileObject.read()

In [15]:
def string_to_nparray_of_bits(s):
    ba = bitarray.bitarray()
    ba.frombytes(data.encode('utf-8'))
    nparray = np.array(list(ba))
    return np.where(nparray == 0, -1, nparray)

In [16]:
def bits_to_string(bs):
    return bitarray.bitarray(list(np.where(bs == -1, 0, bs))).tobytes().decode('utf-8')

In [17]:
bits_to_string(string_to_nparray_of_bits(data))

"Ceci est un test. Je m'appelle Antoine et j'écris des accents pour faire des tests.\n😀\n"

In [10]:
def channel(chanInput):
    chanInput = np.clip(chanInput,-1,1)
    erasedIndex = np.random.randint(3)
    chanInput[erasedIndex:len(chanInput):3] = 0
    return chanInput + np.sqrt(10)*np.random.randn(len(chanInput))

In [11]:
def channel_no_noise(chanInput):
    chanInput = np.clip(chanInput,-1,1)
    erasedIndex = np.random.randint(3)
    chanInput[erasedIndex:len(chanInput):3] = 0
    return chanInput

# Convolutional Encoding

### Encoder

In [18]:
def convolutional_encoder(bs):
    bs = [1, 1] + bs
    codewords = []
    
    for i in range(2, len(bs)):
        codewords.append(bs[i]*bs[i-2])
        codewords.append(bs[i]*bs[i-1]*bs[i-2])
        
    return codewords

In [19]:
def encoder(bs, repeat):
    codewords = convolutional_encoder(bs)
    return np.repeat(codewords, repeat)

### Decoder

In [51]:
def to_dec(alpha):
    return int(str(alpha[0])+str(alpha[1]), 2)

In [57]:
def convolutional_decoder(ys, maxj):
    B_matrix = [[list() for _ in range(4)] for _ in range(maxj)]
    mu_matrix = np.ones((maxj, 2, 2)) * -np.inf
    mu_matrix[0, 1, 1] = 0
    
    for j in range(1, maxj):
        max_cost = -np.inf
        max_beta = None
        for alpha in [(0,0), (0,1), (1,0), (1,1)]:
            for beta in [(0,0), (0,1), (1,0), (1,1)]:
                edge_cost = 0 # TODO
                subpath_cost = mu_matrix[j-1][beta]
                cost = edge_cost + subpath_cost
                if (cost > max_cost):
                    max_cost = cost
                    max_beta = beta
            mu_matrix[j][alpha] = max_cost
            B_matrix[j][to_dec(alpha)] = B_matrix[j-1][to_dec(max_beta)] + None # TODO
    
    return B_matrix[maxj][to_dec((1,1))][:k]

In [62]:
def detect_erasure(dec_ys):
    bins = np.sum(np.abs(dec_ys), axis=1)
    mods = []
    for i in range(3):
        mods[i] = np.sum(bins[i::3])
    return argmin(mods)

In [63]:
def remove_erasures(ys, repeat):
    dec_ys = [ys[i::repeat] for i in range(repeat)]
    mod = detect_erasure(dec_ys)
    return dec_ys[[i % 3 == mod for i in range(repeat)]]

In [64]:
def decoder(ys, repeat):
    yss = remove_erasures(ys, repeat)
    dec = []
    for ys in yss:
        dec.append(convolutional_decoder(ys, len(ys/2) + 2))
    # fais un systeme de vote ?
    return dec[0]

# Plan

- Finir conv. decoder
- Faire des vrais tests avec zero noise
- Faire des vrais tests avec peu noise local
- Faire des vrais tests avec real noise local
- Pipeline server (easy)
- Faire des vrais tests server
- Se poser sur le système de votes